<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/cdvae_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inverse Design of Next-Generation Superconductors Using Data-Driven Deep Generative Models

Reference: https://pubs.acs.org/doi/10.1021/acs.jpclett.3c01260

Author: Kamal Choudhary (kamal.choudhary@nist.gov)

In [1]:
%%time
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q pytorch-lightning wandb torchmetrics==0.6.0 pymatgen==2022.4.26
!pip install -q hydra-core jarvis-tools python-dotenv p-tqdm accelerate

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 50.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.4/329.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# !pip install -q pytorch-lightning==1.3.6

In [3]:
import os
os.chdir('/content')
if not os.path.exists('cdvae'):
  !git clone https://github.com/JARVIS-Materials-Design/cdvae.git

os.chdir('cdvae')
!pip install -e .

Cloning into 'cdvae'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 253 (delta 94), reused 178 (delta 66), pack-reused 0
Receiving objects: 100% (253/253), 146.68 MiB | 19.24 MiB/s, done.
Resolving deltas: 100% (94/94), done.
Updating files: 100% (99/99), done.
Obtaining file:///content/cdvae
  Preparing metadata (setup.py) ... done
  Running setup.py develop for cdvae


In [4]:
import os
if not os.path.exists("/content/cdvae/WABDB"):
  os.makedirs("/content/cdvae/WABDB")
if not os.path.exists("/content/cdvae/HYDRA_JOBS"):
  os.makedirs("/content/cdvae/HYDRA_JOBS")
os.environ["PROJECT_ROOT"]="/content/cdvae"
os.environ["WABDB"]="/content/cdvae/WABDB"
os.environ["WABDB_DIR"]="/content/cdvae/WABDB"
os.environ["HYDRA_JOBS"]="/content/cdvae/HYDRA_JOBS"
%env HYDRA_FULL_ERROR=1


env: HYDRA_FULL_ERROR=1


In [5]:
!echo $HYDRA_FULL_ERROR

1


Data was generated using this [script](https://github.com/JARVIS-Materials-Design/cdvae/blob/main/scripts/generate_data_cdvae.py).

In [6]:

import yaml
import pprint

with open('/content/cdvae/conf/train/default.yaml','r') as f:
  yam = yaml.safe_load(f)
#yam.pop('early_stopping')
#yam['pl_trainer']['fast_dev_run']=True
yam['pl_trainer']['gpus']=0

with open('/content/cdvae/conf/train/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

with open('/content/cdvae/conf/optim/default.yaml','r') as f:
  yam = yaml.safe_load(f)

yam['use_lr_scheduler']=False

with open('/content/cdvae/conf/optim/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

{'deterministic': False,
 'early_stopping': {'patience': '${data.early_stopping_patience}',
                    'verbose': False},
 'model_checkpoints': {'save_top_k': 1, 'verbose': False},
 'monitor_metric': 'val_loss',
 'monitor_metric_mode': 'min',
 'pl_trainer': {'accumulate_grad_batches': 1,
                'fast_dev_run': False,
                'gpus': 0,
                'gradient_clip_algorithm': 'value',
                'gradient_clip_val': 0.5,
                'max_epochs': '${data.train_max_epochs}',
                'num_sanity_val_steps': 2,
                'precision': 32,
                'profiler': 'simple'},
 'random_seed': 42}
{'lr_scheduler': {'_target_': 'torch.optim.lr_scheduler.ReduceLROnPlateau',
                  'factor': 0.6,
                  'min_lr': '1e-4',
                  'patience': 30},
 'optimizer': {'_target_': 'torch.optim.Adam',
               'betas': [0.9, 0.999],
               'eps': '1e-08',
               'lr': 0.001,
               'weight_de

In [7]:
%%time
import os
os.environ["WANDB_ANONYMOUS"] = "must"
!python cdvae/run.py data=supercon expname=supercon_test02 model.predict_property=True

2023-07-31 00:58:45.988087: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 00:58:48.437953: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/cdvae/cdvae/run.py:164: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'default': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-p

In [8]:
!pip install smact

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.4/130.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.7 MB/s eta 0:00:00


In [9]:
# from datetime import date
# d1 = today.strftime("%Y-%m-%d")

In [21]:
temp_dir=!ls /content/cdvae/HYDRA_JOBS/singlerun/

In [26]:
os.environ['TMP_DIR']=temp_dir[0]

In [27]:
!echo $TMP_DIR

2023-07-31


Adjust path accordingly

In [28]:
%%time
!python scripts/evaluate.py --n_step_each 5 --num_batches_to_samples 5 --batch_size 5 --model_path "/content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/supercon_test02" --tasks opt gen recon

2023-07-31 03:01:47.889341: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 03:01:50.070882: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/cdvae/cdvae/pl_data/dataset.py:126: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/cdvae/pl_data/datamodule.py:142: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/

In [12]:
!pip install matminer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 14.0 MB/s eta 0:00:00


In [29]:
%%time
!python scripts/compute_metrics.py --root_path  "/content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/supercon_test02" --tasks   gen recon

2023-07-31 03:32:41.577138: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 03:32:42.991558: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/cdvae/cdvae/pl_data/dataset.py:126: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/cdvae/pl_data/datamodule.py:142: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/

In [38]:
temp_dir[0]

'2023-07-31'

In [35]:
!ls /content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/supercon_test02/

'epoch=139-step=4760.ckpt'   eval_opt.pt     lattice_scaler.pt
 eval_gen.pt		     eval_recon.pt   prop_scaler.pt
 eval_metrics.json	     hparams.yaml    run.log


In [40]:
import torch
from jarvis.core.atoms import Atoms
from jarvis.core.atoms import pmg_to_atoms
from jarvis.core.lattice import Lattice
from jarvis.core.specie import atomic_numbers_to_symbols
from jarvis.db.jsonutils import dumpjson
from jarvis.analysis.structure.spacegroup import Spacegroup3D
from collections import Counter
from pymatgen.core.structure import Structure
import pandas as pd
opt_path = "/content/cdvae/HYDRA_JOBS/singlerun/"+temp_dir[0]+"/supercon_test02/eval_opt.pt"
gen_path = "/content/cdvae/HYDRA_JOBS/singlerun/"+temp_dir[0]+"/supercon_test02/eval_gen.pt"
recon_path = "/content/cdvae/HYDRA_JOBS/singlerun/"+temp_dir[0]+"/supercon_test02/eval_recon.pt"
csv_path = "/content/cdvae/data/supercon/test.csv"
df = pd.read_csv(csv_path)
x = torch.load(recon_path)
y = torch.load(gen_path)
z = torch.load(opt_path)
print(len(df),x["num_atoms"].shape,y["num_atoms"].shape,z["num_atoms"].shape)

105 torch.Size([1, 105]) torch.Size([1, 25]) torch.Size([1, 275])


In [41]:
num_atoms = x["num_atoms"]
atom_types = x["atom_types"]
frac_coords = x["frac_coords"]
lengths = x["lengths"]
angles = x["angles"]
index_list = torch.cumsum(num_atoms[0], dim=0).numpy().tolist()
indice_tuples = []
for i, ii in enumerate(index_list):
    if i == 0:
        tup = [0, index_list[i] - 1]
    else:
        tup = [index_list[i - 1] - 1, index_list[i] - 1]
    indice_tuples.append(tup)

recon_structures = []

for id_needed in range(num_atoms.shape[1]):
    id_fracs = frac_coords[0].numpy()[
        indice_tuples[id_needed][0] : indice_tuples[id_needed][1]
    ]
    id_atom_types = atom_types[0].numpy()[
        indice_tuples[id_needed][0] : indice_tuples[id_needed][1]
    ]
    id_lengths = lengths[0].numpy()[id_needed]
    id_angles = angles[0].numpy()[id_needed]
    lat = Lattice.from_parameters(
        id_lengths[0],
        id_lengths[1],
        id_lengths[2],
        id_angles[0],
        id_angles[1],
        id_angles[2],
    ).matrix
    atoms = Atoms(
        lattice_mat=lat,
        elements=atomic_numbers_to_symbols(id_atom_types),
        coords=id_fracs,
        cartesian=False,
    )
    # spg_numb = Spacegroup3D(atoms).space_group_number
    # spg_numbs.append(spg_numb)

    # print()
    # print()
    # print()
    # print("jarvis\n", atoms)
    # struct = Structure(
    #    lattice=Lat.from_parameters(
    #        id_lengths[0],
    #        id_lengths[1],
    #        id_lengths[2],
    #        id_angles[0],
    #        id_angles[1],
    #        id_angles[2],
    #    ),
    #    species=id_atom_types,
    #    coords=id_fracs,
    #    coords_are_cartesian=False,
    # )
    # atoms = pmg_to_atoms(struct)
    # print("pmg\n", atoms)
    # print()
    # print()
    # print()

    # gen_structures.append(atoms.to_dict())
    recon_structures.append(atoms)


In [42]:
test_structures=[]
jids = []
for i,ii in df.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  test_structures.append(atoms)#.to_dict())
  jids.append(ii['material_id'])

/usr/local/lib/python3.10/dist-packages/pymatgen/io/cif.py:1155: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [43]:
df

,Unnamed: 0,material_id,cif,prop
0,0,JVASP-68993,# generated using pymatgen\ndata_Ba2LiSn\n_sym...,-1.191012
1,1,JVASP-134327,# generated using pymatgen\ndata_CaCO3\n_symme...,-7.781567
2,2,JVASP-102572,# generated using pymatgen\ndata_Ti3Si\n_symme...,-10.491911
3,3,JVASP-16365,# generated using pymatgen\ndata_LiAl3\n_symme...,-0.816908
4,4,JVASP-36657,# generated using pymatgen\ndata_VN\n_symmetry...,-2.649513
...,...,...,...,...
100,100,JVASP-67276,# generated using pymatgen\ndata_Be2NiPt\n_sym...,-0.478394
101,101,JVASP-139460,# generated using pymatgen\ndata_MgTe2\n_symme...,-3.793655
102,102,JVASP-20204,# generated using pymatgen\ndata_Ti3Al\n_symme...,-3.720508
103,103,JVASP-69411,# generated using pymatgen\ndata_Be2CuRh\n_sym...,-1.779018


Uploading to JARVIS-Leaderboard.

In [44]:
from jarvis.io.vasp.inputs import Poscar
import json
f=open('AI-AtomGen-Tc_supercon-dft_3d-test-rmse.csv','w')
line='id,target,prediction\n'
f.write(line)
for i,j,k in zip(test_structures,recon_structures,jids):
  print(k,i.composition.reduced_formula,j.composition.reduced_formula)
  line = k+","+Poscar(i).to_string().replace('\n','\\n')+","+Poscar(j).to_string().replace('\n','\\n')+"\n"
  #line = k+","+json.dumps(i.to_dict())+","+json.dumps(j.to_dict())+"\n"
  f.write(line)
f.close()
#zip file before uploading to JARVIS-Leadrboard

JVASP-68993 Ba2LiSn CuSnTe
JVASP-134327 CaCO3 CaNbTeO2
JVASP-102572 Ti3Si Ti7AsO
JVASP-16365 LiAl3 TiAl3
JVASP-36657 VN AlVN3
JVASP-20620 YB6 SrB5F
JVASP-15923 Ta2N TaBMoN
JVASP-136988 MgP2 MgPN
JVASP-13745 CaSi CaSi3
JVASP-109989 Li3Sn Li3Si2Ag
JVASP-69869 Be2PtRh LiMnBe2
JVASP-129329 NbB BeNbRh
JVASP-78768 FeCo NbCo
JVASP-139524 YAlS3 CoSiS
JVASP-14781 YPd3 ZrZnRuS
JVASP-100039 AlVNi2 Ni2GeRh
JVASP-16459 BeRh AlRh
JVASP-15255 SrAlSi BaSiRh
JVASP-19777 Cr3Ir SrTaCr3MoN2
JVASP-20626 V3Pt V5CrPtRh
JVASP-101611 TaZn3 AlVZn2
JVASP-16521 ScGaNi2 TiGaNi2
JVASP-115500 B2SO2 NiB3SO
JVASP-100364 NbMoO4 NbO5
JVASP-139627 NaTiO2 NaO
JVASP-15075 HfN HfO
JVASP-35497 Y2Fe2Si2C CoTeN
JVASP-72005 BeCo2P Co3S
JVASP-134821 POF BeCoF
JVASP-135523 NaO2 NaO2
JVASP-120382 AsP2 PCO
JVASP-39656 TiNbRe2 TiNbWS
JVASP-78464 TiAl TiNb
JVASP-37463 YNbRu2 AlRu
JVASP-18243 TiSi TiRu
JVASP-40528 LiCa2Ga CaTiGa
JVASP-16432 YAl3 YAl
JVASP-134358 SiCF2 AlCoN
JVASP-16775 HfBeSi CaHf4TiSi4
JVASP-18919 ScRh3 HfRh3
JVASP-1

In [45]:
dfx = pd.read_csv('AI-AtomGen-Tc_supercon-dft_3d-test-rmse.csv')

In [46]:
import pandas as pd
info = {}
test_path = pd.read_csv("/content/cdvae/data/supercon/test.csv")
val_path = pd.read_csv("/content/cdvae/data/supercon/val.csv")
train_path = pd.read_csv("/content/cdvae/data/supercon/train.csv")
test={}
val={}
train={}

for i,ii in train_path.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  pos = Poscar(atoms).to_string().replace('\n','\\n')
  jid=ii['material_id']
  train[jid]=pos

for i,ii in val_path.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  pos = Poscar(atoms).to_string().replace('\n','\\n')
  jid=ii['material_id']
  val[jid]=pos

for i,ii in test_path.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  pos = Poscar(atoms).to_string().replace('\n','\\n')
  jid=ii['material_id']
  test[jid]=pos
info['train']=train
info['val']=val
info['test']=test

/usr/local/lib/python3.10/dist-packages/pymatgen/io/cif.py:1155: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [47]:
from jarvis.db.jsonutils import dumpjson
dumpjson(data=info,filename='dft_3d_Tc_supercon.json')

In [48]:
test_path

,Unnamed: 0,material_id,cif,prop
0,0,JVASP-68993,# generated using pymatgen\ndata_Ba2LiSn\n_sym...,-1.191012
1,1,JVASP-134327,# generated using pymatgen\ndata_CaCO3\n_symme...,-7.781567
2,2,JVASP-102572,# generated using pymatgen\ndata_Ti3Si\n_symme...,-10.491911
3,3,JVASP-16365,# generated using pymatgen\ndata_LiAl3\n_symme...,-0.816908
4,4,JVASP-36657,# generated using pymatgen\ndata_VN\n_symmetry...,-2.649513
...,...,...,...,...
100,100,JVASP-67276,# generated using pymatgen\ndata_Be2NiPt\n_sym...,-0.478394
101,101,JVASP-139460,# generated using pymatgen\ndata_MgTe2\n_symme...,-3.793655
102,102,JVASP-20204,# generated using pymatgen\ndata_Ti3Al\n_symme...,-3.720508
103,103,JVASP-69411,# generated using pymatgen\ndata_Be2CuRh\n_sym...,-1.779018


In [49]:
!cp scripts/compute_metrics.py scripts/eval_utils.py .

In [50]:
!ls

AI-AtomGen-Tc_supercon-dft_3d-test-rmse.csv  env.yml
assets					     eval_utils.py
cdvae					     HYDRA_JOBS
cdvae.egg-info				     LICENSE
compute_metrics.py			     README.md
conf					     requirements.txt
data					     scripts
dft_3d_Tc_supercon.json			     setup.py
env.cpu.yml				     WABDB
env_sub.yml


In [52]:
from p_tqdm import p_map
from compute_metrics import GenEval,get_crystal_array_list,Crystal, RecEval
crys_array_list, true_crystal_array_list = get_crystal_array_list(recon_path)
pred_crys = p_map(lambda x: Crystal(x), crys_array_list)
gt_crys = p_map(lambda x: Crystal(x), true_crystal_array_list)
rec_evaluator = RecEval(pred_crys, gt_crys)
recon_metrics = rec_evaluator.get_metrics()


crys_array_list, _ = get_crystal_array_list(gen_path)
gen_crys = p_map(lambda x: Crystal(x), crys_array_list)
gen_evaluator = GenEval(gen_crys, gt_crys, eval_model_name='carbon')
gen_metrics = gen_evaluator.get_metrics()
print(recon_metrics)

  0%|          | 0/105 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:4133: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:3936: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:4133: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:3936: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(


  0%|          | 0/105 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:4133: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:3936: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:4133: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:3936: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(
100%|██████████| 105/105 [00:00<00:00, 138.98it/s]


  0%|          | 0/25 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/content/cdvae/cdvae/common/data_utils.py:622: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float)
/content/cdvae/eval_utils.py:56: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(str(model_path)):
/usr/local/lib/python3.10/dist-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)
/content/cdvae/eval_utils.py:50: UserWarning: 
The version_base parameter is not specified.
Please specify a 

  0%|          | 0/105 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


{'comp_valid': 0.96, 'struct_valid': 1.0, 'valid': 0.96, 'comp_div': nan, 'struct_div': nan, 'wdist_density': 2.8496816146361774, 'wdist_num_elems': 1.4285714285714286, 'wdist_prop': 0.5526275634765626}
{'match_rate': 0.0380952380952381, 'rms_dist': 0.21668653672743393}


/content/cdvae/cdvae/common/data_utils.py:622: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float)


In [53]:
print(recon_metrics)

{'match_rate': 0.0380952380952381, 'rms_dist': 0.21668653672743393}


In [54]:
print(gen_metrics)

{'comp_valid': 0.96, 'struct_valid': 1.0, 'valid': 0.96, 'comp_div': nan, 'struct_div': nan, 'wdist_density': 2.8496816146361774, 'wdist_num_elems': 1.4285714285714286, 'wdist_prop': 0.5526275634765626, 'cov_recall': 0.0, 'cov_precision': 0.04, 'amsd_recall': 0.5462367326739969, 'amsd_precision': 0.3878688920330222, 'amcd_recall': 9.460229601809257, 'amcd_precision': 7.217619679120109}


In [ ]:
!pip freeze